## Baseline model

#### This notebook is about to do the different baseline model

##### We'll use the original dataset with different model

In [231]:
import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
pd.options.display.max_columns = None

### Load & Split basic data for baseline model

In [68]:
# Will do the baseline with the very basic data
df = pd.read_csv('../data/employee_data.csv')
pd.options.display.max_columns = None
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2


In [69]:
# Covert target
df['Attrition'] = df['Attrition'].map({'Yes':1, 'No':0})

In [70]:
X = df.drop(columns='Attrition')
y = df['Attrition']

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y, test_size=0.15)

In [72]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1249, 34), (221, 34), (1249,), (221,))

### Create Dummies

#### - List all object columns

In [63]:
# Check all object categories
glimpse = []
for c in df.columns:
    make_set = list(set(df[c]))
    data = {'column_name':c, 'set_10':make_set[:10], 'set_count':len(make_set), 'type':str(df[c].dtypes)}
    #glimpse.append(data)
    if data['type'] == 'object':
        glimpse.append(data)
df_glimpse = pd.DataFrame(glimpse)
df_glimpse

,column_name,set_10,set_count,type
0,BusinessTravel,"[Non-Travel, Travel_Rarely, Travel_Frequently]",3,object
1,Department,"[Research & Development, Sales, Human Resources]",3,object
2,EducationField,"[Medical, Human Resources, Other, Life Science...",6,object
3,Gender,"[Female, Male]",2,object
4,JobRole,"[Sales Executive, Manufacturing Director, Labo...",9,object
5,MaritalStatus,"[Divorced, Married, Single]",3,object
6,Over18,[Y],1,object
7,OverTime,"[No, Yes]",2,object


In [94]:
dummy_col = [c for c in df_glimpse['column_name']]

dummy_col

['BusinessTravel',
 'Department',
 'EducationField',
 'Gender',
 'JobRole',
 'MaritalStatus',
 'Over18',
 'OverTime']

In [100]:
X_train_dum = pd.get_dummies(X_train, columns=dummy_col)

X_train_dum.shape

(1249, 55)

In [101]:
X_test_dum = pd.get_dummies(X_test, columns=dummy_col)

X_test_dum.shape

(221, 55)

#### - Baseline model with 4 methods

In [190]:
# function for iterate models
def model_processing(model, X_train, X_test, y_train, y_test):
    scores = []
    cm_train = []
    cm_test = []
    for m in model:
        m.fit(X_train, y_train)
        model_name = type(m).__name__
        scores.append([model_name, m.score(X_train, y_train), m.score(X_test, y_test)])
        
        cm_train.append([model_name, confusion_matrix(y_train, m.predict(X_train))])
        cm_test.append([model_name, confusion_matrix(y_test, m.predict(X_test))])
    return scores, cm_train, cm_test
# Got how to get model name from: https://stackoverflow.com/questions/52763325/how-to-obtain-only-the-name-of-a-models-object-in-scikitlearn

In [191]:
# process all models
lr = LogisticRegression(random_state=42)
rf = RandomForestClassifier(random_state=42)
ada = AdaBoostClassifier(random_state=42)
knn = KNeighborsClassifier()
model = [lr, rf, ada, knn]
scores, cm_train, cm_test = model_processing(model, X_train_dum, X_test_dum, y_train, y_test)

/Users/atigon/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/atigon/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to 

In [212]:
pd.DataFrame(scores, columns=['model', 'train_score', 'test_score'])

,model,train_score,test_score
0,LogisticRegression,0.840673,0.846154
1,RandomForestClassifier,1.000000,0.859729
2,AdaBoostClassifier,0.905524,0.855204
3,KNeighborsClassifier,0.864692,0.782805


In [193]:
# function to print confusion matrix scores
def print_score(cm, is_print=False):
    tn, fp, fn, tp = cm.ravel()
    # calculate accuracy
    conf_accuracy = (float (tp+tn) / float(tp + tn + fp + fn))
    
    # calculate the recall
    conf_recall = (tp / float(tp + fn))

    if is_print:
        decimal = 3
        print(f'Accuracy: \t{round(conf_accuracy,decimal)}') 
        print(f'Recall: \t{round(conf_recall,decimal)}') 
    return conf_accuracy, conf_recall

In [215]:
def confusion_matrix_score(cm, is_print=False):
    cm_scores = []
    for c in cm:
        tn, fp, fn, tp = c[1].ravel()
        acc = (float (tp+tn) / float(tp + tn + fp + fn))
        rcl = (tp / float(tp + fn))
        if is_print:
            print(c[0], ': Accuracy =', acc, ', Recall =', rcl)
        cm_scores.append([c[0], acc, rcl, tn, fp, fn, tp])
    return cm_scores

In [222]:
cm_scores_train = confusion_matrix_score(cm_train)

df_cm_train = pd.DataFrame(cm_scores_train, columns=['model', 'train_accuracy', 'train_recall', 'tn', 'fp', 'fn', 'tp'])
df_cm_train.to_csv('../data/cm_scores_train.csv', index=False)
df_cm_train

,model,train_accuracy,train_recall,tn,fp,fn,tp
0,LogisticRegression,0.840673,0.019900,1046,2,197,4
1,RandomForestClassifier,1.000000,1.000000,1048,0,0,201
2,AdaBoostClassifier,0.905524,0.527363,1025,23,95,106
3,KNeighborsClassifier,0.864692,0.238806,1032,16,153,48


In [223]:
cm_scores_test = confusion_matrix_score(cm_test)

df_cm_test = pd.DataFrame(cm_scores_test, columns=['model', 'test_accuracy', 'test_recall', 'tn', 'fp', 'fn', 'tp'])
df_cm_test.to_csv('../data/cm_scores_test.csv', index=False)
df_cm_test

,model,test_accuracy,test_recall,tn,fp,fn,tp
0,LogisticRegression,0.846154,0.055556,185,0,34,2
1,RandomForestClassifier,0.859729,0.194444,183,2,29,7
2,AdaBoostClassifier,0.855204,0.361111,176,9,23,13
3,KNeighborsClassifier,0.782805,0.027778,172,13,35,1


### Best baseline model is AdaBoostClassifier with recall=0.36 on test data.

In [230]:
df.groupby('OverTime')['Attrition'].mean()

OverTime
No     0.104364
Yes    0.305288
Name: Attrition, dtype: float64

In [89]:
#ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

In [92]:
#X_train_enc = ohe.fit_transform(X_train)

#X_train_enc.shape

(1249, 4810)

In [93]:
#X_test_enc = ohe.transform(X_test)

#X_test_enc.shape

(221, 4810)